# Train and predict with one mechanism 

In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from pandasql import sqldf
from random_profiles import RandomProfileGenerator

pd.options.display.max_rows = 5
pd.options.display.max_columns = 300
%matplotlib inline

pqry = lambda q: sqldf(q, globals())

In [2]:
# get data
data = pd.read_excel('Data/Final_Berg JBS 2013 Supplemental Table 3_For SVM14Dec2017 - Labels.xlsx')

# Select Mechanism with greatest count 
pqry('''SELECT Mechanism, count(Mechanism) count FROM data m group by Mechanism order by count(Mechanism) desc LIMIT 1;''')


,Mechanism,count
0,mTOR inhibitor,27


In [3]:
# Impute missing
grouped = data.iloc[:,2:].groupby(data["Mechanism"])
f = lambda x: x.fillna(x.mean())
data.iloc[:,2:] = grouped.transform(f)
#data[75:82]

In [4]:
mechanisms = pqry('''SELECT distinct Mechanism from data''').values.ravel()

In [5]:
pos_class = pqry('''select * from data where Mechanism = "mTOR inhibitor" ''' ).iloc[:, 1:]

In [14]:
# Initialize the random profile genearator with sig env values and profile data file for S:M
rpg = RandomProfileGenerator(envelope_file='SigEnvelopeFile.xml', data_file='SVMData.xlsx')

envelope=RandomProfileGenerator.TRUSTED 
conf=RandomProfileGenerator._95
neg_values = rpg.get_random_profiles(prof_num=len(pos_class), envelope=envelope,conf=conf, dist='rand')


In [13]:
neg_mech = pd.DataFrame({'Mechanism': ['neg_class' for s in range(len(pos_class))]})

In [29]:
neg_class = pd.concat([neg_mech, neg_values], axis=1)
all_class = pd.concat([pos_class, neg_class]).reset_index(drop=True)
X = all_class.iloc[:,1:]
Y = all_class.iloc[:,0]
Y

0     mTOR inhibitor
1     mTOR inhibitor
           ...      
52         neg_class
53         neg_class
Name: Mechanism, Length: 54, dtype: object

In [ ]:
# test train split